# 1. 연관분석 개요

- 데이터들 사이에서 자주 발생하는 속성을 찾고 그 속성들 사이에 
  연관성이 어느 정도 있는지를 분석하는 방법
- 활용분야 : 상품진열, 사기보험적발, 카탈로그 디자인, 신상품 카테고리 구성,...
- 연관성분석 관련 지표 
<pre>   
         지지도(support) : 조건결과항목수/전체수
         신뢰도(Confidence) : 조건결과항목수/ 조건항목수
         향상도(lift) : 우연히 발생한 규칙인지 아닌지 여부
                        = 조건결과 지지도 / (조건지지도)*(결과지지도)
                            1: 상관관계없음
                            1이상: 양의상관관계
                            1이하: 음의 상관관계               
        [조건] => [결과]  지지도 신뢰도 향상도
        [주스] => [콜라]   0.4     1     0.4 / (0.4)*1 = 1
        [소주] => [맥주]   0.2    0.33   0.2 / (0.6)*(0.4)
</pre>

In [3]:
# 트랜잭션 데이터 가져오기
import csv
with open('cf_basket.csv','r',encoding='utf8') as cf:
    transactions=[]
    r = csv.reader(cf)
    for row in r:
        transactions.append(row)

In [4]:
transactions

[['소주', '콜라', '와인'],
 ['소주', '오렌지주스', '콜라'],
 ['콜라', '맥주', '와인'],
 ['소주', '콜라', '맥주'],
 ['오렌지주스', '와인', '콜라']]

# 2. 연관분석
- pip install apyori

In [29]:
from apyori import apriori
rules = apriori(transactions,min_support=0.2,min_confidence=0.1)
results=list(rules)

In [28]:
print('조건    ->   결과 \t\t지지도\t신뢰도\t향상도')
for row in results:
    support = row[1]
    ordered_st = row[2]
    for item in ordered_st:
        lhs = [x for x in item[0]]
        rhs = [x for x in item[1]]
        confidence = item[2]
        lift = item[3]
        if lift != 1:
            print(lhs,' => ',rhs,'\t\t{:5.3f}\t{:5.3f}\t{:5.3f}'.format(support,confidence,lift))

조건    ->   결과 		지지도	신뢰도	향상도
['맥주']  =>  ['소주'] 		0.200	0.500	0.833
['소주']  =>  ['맥주'] 		0.200	0.333	0.833
['맥주']  =>  ['와인'] 		0.200	0.500	0.833
['와인']  =>  ['맥주'] 		0.200	0.333	0.833
['소주']  =>  ['오렌지주스'] 		0.200	0.333	0.833
['오렌지주스']  =>  ['소주'] 		0.200	0.500	0.833
['소주']  =>  ['와인'] 		0.200	0.333	0.556
['와인']  =>  ['소주'] 		0.200	0.333	0.556
['오렌지주스']  =>  ['와인'] 		0.200	0.500	0.833
['와인']  =>  ['오렌지주스'] 		0.200	0.333	0.833
['맥주']  =>  ['콜라', '소주'] 		0.200	0.500	0.833
['소주']  =>  ['콜라', '맥주'] 		0.200	0.333	0.833
['콜라', '맥주']  =>  ['소주'] 		0.200	0.500	0.833
['콜라', '소주']  =>  ['맥주'] 		0.200	0.333	0.833
['맥주']  =>  ['콜라', '와인'] 		0.200	0.500	0.833
['와인']  =>  ['콜라', '맥주'] 		0.200	0.333	0.833
['콜라', '맥주']  =>  ['와인'] 		0.200	0.500	0.833
['콜라', '와인']  =>  ['맥주'] 		0.200	0.333	0.833
['소주']  =>  ['콜라', '오렌지주스'] 		0.200	0.333	0.833
['오렌지주스']  =>  ['콜라', '소주'] 		0.200	0.500	0.833
['콜라', '소주']  =>  ['오렌지주스'] 		0.200	0.333	0.833
['콜라', '오렌지주스']  =>  ['소주'] 		0.200	0.500	0.833
['소주']  =>  ['콜라',

# 3. 뉴스기사 연관 분석 실습

## 3.1 뉴스 RSS 이용해서 기사 검색후 연관분석

In [35]:
import requests
from bs4 import BeautifulSoup 
rss_url ='https://rss.joins.com/joins_money_list.xml'
money_response = requests.get(rss_url)
money_soup = BeautifulSoup(money_response.content,'xml')
link_list = money_soup.select('item > link')
link_list

[<link>https://news.joins.com/article/24021331?cloc=rss-news-economy</link>,
 <link>https://news.joins.com/article/24021323?cloc=rss-news-economy</link>,
 <link>https://news.joins.com/article/24021317?cloc=rss-news-economy</link>,
 <link>https://news.joins.com/article/24021309?cloc=rss-news-economy</link>,
 <link>https://news.joins.com/article/24021300?cloc=rss-news-economy</link>,
 <link>https://news.joins.com/article/24021294?cloc=rss-news-economy</link>,
 <link>https://news.joins.com/article/24021290?cloc=rss-news-economy</link>,
 <link>https://news.joins.com/article/24021287?cloc=rss-news-economy</link>,
 <link>https://news.joins.com/article/24021263?cloc=rss-news-economy</link>,
 <link>https://news.joins.com/article/24021234?cloc=rss-news-economy</link>,
 <link>https://news.joins.com/article/24021228?cloc=rss-news-economy</link>,
 <link>https://news.joins.com/article/24021164?cloc=rss-news-economy</link>,
 <link>https://news.joins.com/article/24021110?cloc=rss-news-economy</link>,

In [47]:
from konlpy.tag import Kkma
kkma = Kkma()
news = []
for link in link_list:
    news_response = requests.get(link.text)
    news_soup = BeautifulSoup(news_response.content,'html.parser')
    content = news_soup.select_one('#article_body').text
    temp = list(filter(lambda word : len(word)>1,kkma.nouns(content)))
    news.append(temp)

In [49]:
# 연관분석
from apyori import apriori
rules = apriori(news, 
                min_support=0.3,
                min_confidence=0.2)
result = list(rules)

In [54]:
import pandas as pd
result_df = pd.DataFrame(None,columns=['lhs','rhs','support','confidence','lift'])
index=0
for row in result:
    support = row[1]
    order_st = row[2]
    for item in order_st:
        lhs = ','.join(x for x in item[0])
        rhs = ','.join(x for x in item[1])
        confidence = item[2]
        lift = item[3]
        if lift != 1:
            result_df.loc[index]=[lhs,rhs,support,confidence,lift]
            index+=1

0      1.173913
1      1.173913
2      1.666667
3      1.666667
4      0.984848
         ...   
729    3.333333
730    3.333333
731    3.000000
732    3.333333
733    3.000000
Name: lift, Length: 734, dtype: float64

In [70]:
pd.options.display.max_rows = 644
result_df.loc[(result_df.lhs.str.contains('기자'))&(result_df.rhs.str.contains('한국'))].sort_values(by=['lift'],ascending=False)

,lhs,rhs,support,confidence,lift
329,"관련,기자",한국,0.333333,0.555556,1.111111
327,기자,"관련,한국",0.333333,0.434783,1.086957
112,기자,한국,0.366667,0.478261,0.956522


In [88]:
# 원하는 내용 크롤링하여 연관분석
url ='https://search.daum.net/search?nil_suggest=btn&w=tot&DA=SBC&q=%EC%BD%94%EB%A1%9C%EB%82%98'
corona = requests.get(url)
corona_soup = BeautifulSoup(corona.content,'html.parser')
link_list = corona_soup.select('#clusterResultUL .f_link_b')
links =[]
for link in link_list:
    links.append(link.get('href'))
links

['http://v.media.daum.net/v/20210326174916639?f=o',
 'http://v.media.daum.net/v/20210326171857722?f=o',
 'http://v.media.daum.net/v/20210326174754585?f=o',
 'http://v.media.daum.net/v/20210326172955032?f=o']

In [99]:
news = []
for link in links:
    news_response = requests.get(link)
    news_soup = BeautifulSoup(news_response.content,'html.parser')
    news_content = news_soup.select('#news_body_id .par')
#### 링크마다 사이트 구성이 달라서 막힌다
### 이런 연유로 여러 뉴스사이트의 데이터를 크롤링할때, 1차적으로 각 뉴스사이트 마다
### 뉴스를 따로 취합하여 2차에 합치는 방법을 쓴다.

[]